In [1]:
# Pricing Tool Change Log

# 2018-08-08 12:48 [JMC]
# - Added the support for pandas
# - Optimization of the code to avoid calling the same function more than once
# - Added support for excel file                  

In [2]:
# Libraries needed for all the code
from __future__ import division   
import numpy as np
from math import exp, sqrt, log
import datetime
from scipy.interpolate import interp1d
from scipy.special import erf, erfinv
import pandas as pd                                           

In [3]:
# Total number of payments in the loan life
# freq: payment frequency (1 - monthly, 3 - quarterly, 6 - semi-annual, 12 - annualy)
# T: maturity in years
def NumberofPayment(freq, T):
    n = int((float(T) / float(freq)) * 12)
    return n      

In [4]:
# Calculates the amortisation
# N: initial notional value
# NP: number of payments
def Amortisation(N, NP):
    amort = np.zeros(NP)
    for i in range(NP):
        amort[i] = float(N) / float(NP)
    return amort.tolist()       

In [5]:
# Calculates the notional value
# N: initial notional value
# NP: number of payments
# Amortisation: amortisation vector
def Notional_Value(N, NP, Amortisation):
    cf = []
    AA = 0     
    for i in range(NP):
        cf.append(N - AA)
        AA = Amortisation[i] + AA
    return cf      

In [6]:
# Cubic Interpolation to derive the forward rate
# freq: payment frequency (1 - monthly, 3 - quarterly, 6 - semi-annual, 12 - annualy)
# Indexant: interst rate index associated to the loan (3 - Euribor 3M, 6 - Euribor 6M, 12 - Euribor 12M)
# tenor: time vector in years related with the swap rates
# swap_rate: vector with the values of the swap curve
# NP: number of payments

def Rate_Aux(freq, Indexant, tenor, swap_rate, NP):
    len_sw_rt = len(swap_rate)
    zero_rate = np.zeros(len_sw_rt)
    count = 0
    for jj in range(len_sw_rt - 1):
        if tenor[jj] < 1:
            zero_rate[jj] = swap_rate[jj]
        elif tenor[jj] == 1:
            zero_rate[jj] = swap_rate[jj]
            count = jj + 1
        else:
            plus = 0
            for i in range(count, len_sw_rt, 1):
                plus = plus + exp(-zero_rate[i - 1])
                Add = swap_rate[i] * plus
                zero_rate[i] = (-log((1 - Add) / (1 + swap_rate[i])))/tenor[i]

    f2 = interp1d(tenor, zero_rate, kind = 'cubic')   
    xnew = np.linspace(tenor[0], tenor[len_sw_rt - 1], endpoint = True)    
    FREQ = float(12)/ float(1)        
    range_rate = int(tenor[len_sw_rt - 1] * FREQ)
    A_int_zero_rate = np.zeros(range_rate)
    new_tenor = np.zeros(range_rate)
    for value in range(range_rate):
        new_tenor[value] = float(value + 1) / float(12)
    k = 0
    T = 12        
    for value in range(range_rate):
        if value != 0:       
             A_int_zero_rate[value]=f2(float(value)/float(T))                       

    return A_int_zero_rate.tolist()



def Forward_Rate(freq, T, A_int_zero_rate, Indexant, NP):
    forward_rate = []   
    if NP + Indexant < 50 * 12:    
        for value in range(T*12):    
             if value % freq == 0:
                forward_rate.append(exp((A_int_zero_rate[value + Indexant] - A_int_zero_rate[value]) * 
                                        float(12)/float(Indexant)) -1)   
    #return forward_rate.tolist()  
    return forward_rate  


def Discount_Rate(freq, T, A_int_zero_rate, NP):  
    discount_rate = []
    if NP + 1 < 50 * 12:   
        for value in range(T*12):    
                if value % freq == 0:        
                        discount_rate.append(exp((A_int_zero_rate[value + freq] - A_int_zero_rate[value])* 
                                                 float(12)/float(freq)) -1) 
    #return discount_rate.tolist()
    return discount_rate     

In [7]:
# Calculate the cash fllow installment
# N: initial notional value
# Spread: spread of the contract
# fixed_rate
# type_of_coupon
# redemption_rate: the early redemption rate factor
# NP: number of payments
# Amortisation: amortisation vector
# forward_rate: forward rate vector

def Cashflow_Instalment(freq, N, funding_cost, hedging_cost, operational_cost, Spread, fixed_rate, 
                        type_of_coupon, NP, forward_rate):
    Cashflow = np.zeros(NP)
    Cashflow_hc = np.zeros(NP)  
    fr = np.zeros(NP)
    hc = np.zeros(NP)
    oc = np.zeros(NP)   
    fc = np.zeros(NP)
    coupon_hc = np.zeros(NP)
    coupon_oc = np.zeros(NP)    
    coupon = np.zeros(NP)      
    deltaT = float(freq)/float(12)
    for i in range(NP):
        fr[i] = fixed_rate
        hc[i] = hedging_cost
        oc[i] = operational_cost
        fc[i] = funding_cost
        if type_of_coupon > 0:
            coupon[i] = (forward_rate[i] + Spread) * deltaT
            coupon_hc[i] = 0   
            coupon_oc[i] = (forward_rate[i] + Spread - oc[i]) * deltaT          
        else:
            coupon[i] = fr[i] * deltaT 
            coupon_hc[i] = (fr[i] - hc[i]) * deltaT   
            coupon_oc[i] = (fr[i] - oc[i]) * deltaT           
    return coupon, coupon_hc, coupon_oc          

def Cashflow(N, Amortisation, redemption_rate, coupon, NP): 
    Cashflow = np.zeros(NP)    
    RF = 1 - redemption_rate 
    AA = 0  
    for i in range(NP):  
        Cashflow[i] = Amortisation[i] + (N - AA) * RF * coupon[i]    
        AA = Amortisation[i] + AA               
    return Cashflow.tolist() 

def Cashflow_hc(N, type_of_coupon, Amortisation, redemption_rate, coupon_hc, NP): 
    if type_of_coupon == 0:
        Cashflow_hc = np.zeros(NP)  
        RF = 1 - redemption_rate  
        AA = 0                                            
        for i in range(NP):  
            Cashflow_hc[i] = Amortisation[i] + (N - AA) * RF * coupon_hc[i]     
            AA = Amortisation[i] + AA  
        return Cashflow_hc.tolist() 
    else: 
        return 0  
    
    
def Cashflow_oc(N, Amortisation, redemption_rate, coupon_oc, NP):   
    Cashflow_oc = np.zeros(NP)     
    RF = 1 - redemption_rate 
    AA = 0  
    for i in range(NP):  
        Cashflow_oc[i] = Amortisation[i] + (N - AA) * RF * coupon_oc[i]    
        AA = Amortisation[i] + AA               
    return Cashflow_oc.tolist()    

In [8]:
# Calculate the interest rate
# N: initial notional value
# Spread: spread of the contract
# fixed_rate
# type_of_coupon
# redemption_rate: the early redemption rate factor
# NP: number of payments
# forward_rate: forward rate vector
# Amortisation: amortisation vector

def Interest_Payment(freq, N, Spread, fixed_rate, type_of_coupon, redemption_rate, NP, Amortisation, forward_rate):
    Cashflow = []
    fr = np.zeros(NP)
    coupon = np.zeros(NP)  
    RF = 1 - redemption_rate
    deltaT = float(freq)/float(12)
    for value in range(NP):
        fr[value] = fixed_rate
        if type_of_coupon > 0:
            coupon[value] = (forward_rate[value] + Spread) * deltaT
        else:
            coupon[value] = fr[value] * deltaT
    AA = 0
    for i in range(NP):   
        Cashflow.append((N - AA) * RF * coupon[i]) 
        AA = Amortisation[i] + AA
    return Cashflow      

In [9]:
# Calculate de probability of default according to the risk class and the probability matrix
def PD(prob_matrix, KK):
    return prob_matrix[KK][len(prob_matrix)]     

In [10]:
# Calculate the loan value (percentage - the amount has to be multiplied by the notional loan amount)
# freq: payment frequency (1 - monthly, 3 - quarterly, 6 - semi-annual, 12 - annualy)
# LGD: loss given default
# KK: risk_class (classification grade - zero based)
# prob_matrix: probability matrix (number of risk classes x number of risk classes + 1)
# FC: funding cost spread
# NP: number of payments
# discount_rate: discount rate vector
# NV: notional value vector
# CI: cashflow instalment vector
# IP: interest payment vector

def Final_LoanPrice(freq, LGD, KK, prob_matrix, FC, NP, discount_rate, NV, CI, IP):

    m = len(prob_matrix)
    deltaT = float(freq) / float(12)
    R = 1 - LGD

    LoanPrice = np.zeros((NP+1, m))    
    LoanPrice[:][-1:] = CI[NP-1]     
      
    for i in range(NP - 1, -1, -1):
        ci_val = 0         
        if i != 0:
            ci_val = CI[i-1]             
        for k in range(m):
            Probability_Default = prob_matrix[k][m] * R * (NV[i] + IP[i])           
            Add = sum(LoanPrice[i + 1][j] * prob_matrix[k][j] for j in range(m))       
            LoanPrice[i][k] = exp(-((discount_rate[i] + FC) * deltaT)) * (Add + Probability_Default) + ci_val         
          
    return LoanPrice[0][KK]           

In [11]:
# Calculate the loan value without risk
# freq: payment frequency (1 - monthly, 3 - quarterly, 6 - semi-annual, 12 - annualy)
# KK: risk_class (classification grade - zero based)
# prob_matrix: probability matrix (number of risk classes x number of risk classes + 1)
# NP: number of payments
# discount_rate: discount rate vector
# CI: cashflow instalment vector

def Final_LoanPrice_WR(freq, KK, prob_matrix, FC, NP, discount_rate, CI):
    m = len(prob_matrix) 
    deltaT = float(freq) / float(12)
    
    diagonal = np.diag(prob_matrix)  
    diagonal_matrix = np.diag(diagonal)
    
    LoanPrice = np.zeros((NP+1, m))
    LoanPrice[:][-1:] = CI[NP - 1]
   
    for i in range(NP - 1, -1, -1):
        ci_val = 0
        if i != 0:
             ci_val = CI[i-1]   
        for k in range(m): 
            Add = LoanPrice[i + 1][k]     
            LoanPrice[i][k] = exp(-((discount_rate[i] + FC) * deltaT)) * Add + ci_val   

    return LoanPrice[0][KK]    

In [12]:
# here is the calculation of the "hedging cost amount" whiich is different than the hedging cost percentage  

def Final_LoanPrice_WR_HC(freq, KK, prob_matrix, FC, hc, NP, discount_rate, CI, type_of_coupon):
    m = len(prob_matrix) 
    deltaT = float(freq) / float(12)
    
    if type_of_coupon == 0: 
        diagonal = np.diag(prob_matrix)  
        diagonal_matrix = np.diag(diagonal)  
    
        LoanPrice = np.zeros((NP+1, m))
        LoanPrice[:][-1:] = CI[NP - 1]
   
        for i in range(NP - 1, -1, -1):
                ci_val = 0
                if i != 0:
                     ci_val = CI[i-1]   
                for k in range(m): 
                        Add = LoanPrice[i + 1][k]     
                        LoanPrice[i][k] = exp(-((discount_rate[i] + FC) * deltaT)) * Add + ci_val  

        return LoanPrice[0][KK]
    
    else: 
             return 0  

In [13]:
# here is the calculation of the "funding cost amount" by using the loan value without risk 

def Final_LoanPrice_WFC(freq, KK, prob_matrix, FC, NP, discount_rate, CI):
    m = len(prob_matrix) 
    deltaT = float(freq) / float(12)
    
    diagonal = np.diag(prob_matrix)  
    diagonal_matrix = np.diag(diagonal)
    
    LoanPrice = np.zeros((NP+1, m))
    LoanPrice[:][-1:] = CI[NP - 1]
   
    for i in range(NP - 1, -1, -1):
        ci_val = 0
        if i != 0:
             ci_val = CI[i-1]              
        for k in range(m): 
            Add = LoanPrice[i + 1][k]     
            LoanPrice[i][k] = exp(- discount_rate[i] * deltaT) * Add + ci_val  
            
    return LoanPrice[0][KK]                          

In [14]:
# here is the calculation of the "operational cost amount" by using the loan value without risk   

def Final_LoanPrice_OC(freq, KK, prob_matrix, FC, OC, NP, discount_rate, CI):
    m = len(prob_matrix) 
    deltaT = float(freq) / float(12)     
         
    diagonal = np.diag(prob_matrix)  
    diagonal_matrix = np.diag(diagonal)
    
    LoanPrice = np.zeros((NP+1, m))
    LoanPrice[:][-1:] = CI[NP - 1]
   
    for i in range(NP - 1, -1, -1):
        ci_val = 0
        if i != 0:
             ci_val = CI[i-1]   
        for k in range(m): 
            Add = LoanPrice[i + 1][k]     
            LoanPrice[i][k] = exp(-((discount_rate[i] + FC) * deltaT)) * Add + ci_val
    
    return  LoanPrice[0][KK] 

In [15]:
# Calculate the hedging cost
# freq: payment frequency (1 - monthly, 3 - quarterly, 6 - semi-annual, 12 - annualy)
# NV: notional value vector
# FR: forward rate vector
# Spread: spread of the contract
# NP: number of payments

def HedgingCost(freq, NV, forward_rate, Spread, NP, type_of_coupon):
    Time = np.zeros(NP)
    tau = np.zeros(NP)
    discount_curve = np.zeros(NP)
    deltaT = float(freq) / float(12)
    if type_of_coupon == 0: 

        for i in range(NP):
            if i == 0:
                    discount_curve[i] = exp(-forward_rate[i] * deltaT)
 
            else:
                    discount_curve[i] = exp(sum(-forward_rate[j] * deltaT for j in range(i + 1)))

            
        Add0 = sum(NV[j] * (forward_rate[j] * deltaT) * discount_curve[j] for j in range(NP))
        Add1 = sum(NV[j] * deltaT * discount_curve[j] for j in range(NP))     
        Add = float(Add0) / float(Add1) 
        return Add 
    
    else: 
        
            return 0    

In [16]:
# Calculate the Economic Capital
# PD: probability of default
# N: initial notional value
# LGD: loss given default
# rho: asset correlation 
# aa:  amount of normal inverse(0.999) 

def EconomicCapital(PD, N, LGD, rho, aa):
    # Normal Inverse
    NI = sqrt(2) * erfinv(2 * PD - 1)

    # Normal
    x = (NI + sqrt(rho) * aa) / sqrt(1 - rho) - PD
    NN = float(1) / float(2) + float(1) / float(2) * erf(x / sqrt(2))

    # Economic Capital
    return N * LGD * NN   

In [17]:
# Calculate the unexpected loss here is the unexpected loss calculation
# EC: economic capital
# wt: target return on the equity capital 
# wr: return, namely invested on the govermental bond 
# Nd: total notional amount 
def Unexpectedloss(EC, wt, wr, Nd):  
    return (wt - wr) * EC / Nd

# Here we call the return of this function as Value_SU  

In [18]:
# Calculate de expected loss
def Final_ExpectedLoss(freq, LGD, KK, prob_matrix, FC, discount_rate, NV, IP, NP):
    deltaT = float(freq) / float(12)    
    m = len(prob_matrix)    

    ExpectedLoss = np.zeros((NP+1, m))
    ExpectedLoss[:][-1:] = 0
 
    for i in range(NP - 1, -1, -1): 
        for k in range(m):     
            Probability_Default = prob_matrix[k][m] * LGD * (NV[i] + IP[i])   
            Add = sum(ExpectedLoss[i + 1][j] * prob_matrix[k][j] for j in range(m))      
            ExpectedLoss[i][k] = exp(-((discount_rate[i] + FC) * deltaT)) * (Add + Probability_Default)    
   
    return ExpectedLoss[0][KK]    

In [19]:
# Auxiliary functions to help the calculation of the interest rate return of EL, UL and HC

def f(x, freq, Value, CI, NP):
    a = float(freq) / float(12)
    Sum = sum(CI[j] * exp(-x * (j + 1) * a) for j in range(NP))
    return Value - Sum

def bisection(a, b, freq, Value, CI, NP, tol = 0.1, maxiter = 10):
    n = 1
    while n <= maxiter:
        c = (a + b) * 0.5
        f_val = f(c, freq, Value, CI, NP)
        if f_val == 0 or abs(a - b) * 0.5 < tol:
            return c
        n += 1
        if f_val < 0:
            a = c
        else:
            b = c
    return c

In [20]:
def SpreadOf(IRR_x, IRR_WR):
    return IRR_x - IRR_WR

In [21]:
# here is the calculation of the "hurdle rate" and the "RAROC formula"   

def RAROC(z, IRR_WFC, Spread, fixed_rate, type_of_coupon, HC_rate, FC_spread, EL_spread, OC_spread, UL_spread, EC, N, wr):
    
    if type_of_coupon > 0:
        z = IRR_WFC + Spread 
    else:
        z = fixed_rate    
    
    hurdle_rate = HC_rate + FC_spread + EL_spread + UL_spread + OC_spread 
    a = z - HC_rate - FC_spread - EL_spread - OC_spread 
    b = EC / N
    RAROC = a / b + wr
    return hurdle_rate, RAROC         

# z in the case of the fixed rate is the fixed rate
# but z in the case of floating rate is the something plus spread 
# it has to be interest rate return of the loan without funding cost 
# plus spread 

In [22]:
#####################################################################################
# CONSTANTS
# This section includes the values that will never change in the algorithm
#####################################################################################

# Initial notional value will be 1 for all cases.
# The real output amounts of the algorithm will need to be multiplied 
# by the loan amount inserted by the user.
#const_init_notional_val = 1

# Aux constant values for the calculation of Economic Capital and Unexpected Loss
#const_rho = 0.25
#const_aa = 3.090232
#const_wt = 0.12
#const_wr = 0.02
#const_nd = 1

In [23]:
#####################################################################################
# CONFIGURATION
# This section includes the inputs of the algorithm that will be configured
# by the user and not directly inserted
#####################################################################################

# Time vector in years (for swap rates)
inp_tenor = [
    float(1) / float(365), float(1) / float(48), float(2) / float(48), float(1) / float(12),
    float(2) / float(12), float(3) / float(12), float(6) / float(12), float(9) / float(12),
    1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
    11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
    21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
    40, 50]

# Swap Rates Values
inp_swap_rates = [
    -0.0047000, -0.0037900, -0.0037100, -0.0037000,
    -0.0033900, -0.0032100, -0.0026800, -0.0021400,
    -0.0018300, -0.0017000, -0.0002000, 0.0014090, 0.0029300, 0.0043800, 0.0057300, 0.0069900, 0.0081500, 0.0091800,
    0.0100100, 0.0109400, 0.0115210, 0.0121590, 0.0128400, 0.0131550, 0.0135900, 0.0135900, 0.0141750, 0.0145400,
    0.0147200, 0.0147200, 0.0148450, 0.0147200, 0.0150720, 0.0150800, 0.0151200, 0.0151600, 0.0151800, 0.0153100,
    0.0153000, 0.0151000]

# Probability Matrix


inp_probability_matrix = [
    [0.9700, 0.0150, 0.0070, 0.0040, 0.0020, 0.0010, 0.0007, 0.0003], 
    [0.0150, 0.9598, 0.0100, 0.0080, 0.0040, 0.0015, 0.0010, 0.0007],
    [0.0100, 0.0200, 0.9272, 0.0200, 0.0100, 0.0070, 0.0040, 0.0018], 
    [0.0050, 0.0100, 0.0200, 0.8905, 0.0400, 0.0200, 0.0100, 0.0045],
    [0.0030, 0.0070, 0.0150, 0.0400, 0.8700, 0.0350, 0.0200, 0.0100],
    [0.0025, 0.0075, 0.0150, 0.0250, 0.0400, 0.8400, 0.0500, 0.0200],
    [0.0020, 0.0050, 0.0100, 0.0150, 0.0350, 0.0700, 0.8130, 0.0500]
     ]  



const_init_notional_val = 1

# Aux constant values for the calculation of Economic Capital and Unexpected Loss
const_rho = 0.25
const_aa = 3.090232
const_wt = 0.12
const_wr = 0.02
const_nd = 1




# Funding Cost Spread
inp_funding_cost_spread = 0.005

# Operating Costs Spread
inp_operation_cost_spread = 0.005

# Early Redemption Rate    # RF is the redemption factor which is: RF = 1 - redemption 
inp_redemption_rate = 0.02  


# Loan Amount
inp_loan_amount = 100000

# Payment Frequency (1 - monthly, 3 - quarterly, 6 - semi-annual, 12 - annualy)
inp_freq = 6

# Maturity (in years)
inp_maturity = 5      

# Indexant (3 - Euribor 3M, 6 - Euribor 6M, 12 - Euribor 12M)
inp_indexant = 3

# Fixed Interest Rate (when applicable)
inp_fixed_rate = 0.02

# Spread of the loan
inp_loan_spread = 0.01

# Type of Coupon (0 - fixed rate, 1 - float rate)
inp_type_coupon = 1

# Loss Given Default
inp_lgd = 0.5 

# Risk Class (zero based)   
inp_risk_class = 6 

# by the loan amount inserted by the user.   
const_init_notional_val = 1      

In [24]:
####################################
# CALCULATION                      #
####################################

# Record the date and time of the beginning of the process
test_start_date = datetime.datetime.now()   



##########################################################

# Calculate the total number of payments
out_num_payments = NumberofPayment(inp_freq, inp_maturity)  
print ('Number of Payments:', out_num_payments)      
    

# Calculate the amortisation
out_amortisation = Amortisation(const_init_notional_val, out_num_payments)
#print ('Amortisation:', out_amortisation)
  

# Calculate the notional value in time
out_notional_value = Notional_Value(const_init_notional_val, out_num_payments, out_amortisation)
#print ('Notional Value:', out_notional_value)


#######################################################


# Calculate the int zero rate (aux for discount and forward rates)
out_int_zero_rate = Rate_Aux(inp_freq, inp_indexant, inp_tenor, inp_swap_rates, out_num_payments)
#print ('Zero Rate (aux):', out_int_zero_rate)


# Calculate the forward rate
out_forward_rate = Forward_Rate(inp_freq, inp_maturity, out_int_zero_rate, inp_indexant, out_num_payments)
#print ('Forward Rate:', out_forward_rate)  

# Calculate the discount rate
out_discount_rate = Discount_Rate(inp_freq, inp_maturity, out_int_zero_rate, out_num_payments)
#print ('Discount Rate:', out_discount_rate)
     
########################################################        

# Calculate the hedging cost (percentage)
out_hedging_cost_perc = HedgingCost(inp_freq, out_notional_value, out_forward_rate, inp_loan_spread, out_num_payments,
                                   inp_type_coupon)    
#print ('Hedging Cost (percentage):', out_hedging_cost_perc)    
      


 # Calculate the cashflow instalments
out_coupon, out_coupon_hc, out_coupon_oc = Cashflow_Instalment(inp_freq, const_init_notional_val, 
                                                                              inp_funding_cost_spread, out_hedging_cost_perc, inp_operation_cost_spread, inp_loan_spread, inp_fixed_rate,
                                                                              inp_type_coupon, out_num_payments, out_forward_rate)
#print ('Cash Flow Instalment:', out_coupon_oc) 



out_cashflow = Cashflow(const_init_notional_val, out_amortisation, inp_redemption_rate
                                      , out_coupon, out_num_payments)  
#print ('Cash Flow:', out_cashflow)



out_cashflow_hc = Cashflow_hc(const_init_notional_val, inp_type_coupon, out_amortisation, inp_redemption_rate
                                      , out_coupon_hc, out_num_payments)  
#print ('Cash Flow_hedging_cost:', out_cashflow_hc)


out_cashflow_oc = Cashflow_oc(const_init_notional_val, out_amortisation, inp_redemption_rate
                                      , out_coupon_oc, out_num_payments)  
#print ('Cash Flow_hedging_cost:', out_cashflow_oc)
 



# Calculate the interest payment
out_interest_payment = Interest_Payment(inp_freq, const_init_notional_val, inp_loan_spread, inp_fixed_rate,
                                        inp_type_coupon, inp_redemption_rate, out_num_payments,
                                        out_amortisation, out_forward_rate)
#print ('Interest Payment:', out_interest_payment)

##################################################################


# Calculate the loan value
out_loan_value_perc = Final_LoanPrice(inp_freq, inp_lgd, inp_risk_class, inp_probability_matrix, 
                                      inp_funding_cost_spread, out_num_payments,
                                      out_discount_rate, out_notional_value, out_cashflow, out_interest_payment)
#print ('Loan Value (percentage):', out_loan_value_perc)

# Calculate the loan value amount
out_loan_value_amt = out_loan_value_perc * inp_loan_amount
#print ('Loan Value Amount:', out_loan_value_amt)

# Calculate the return of the loan
out_loan_return = bisection(-10, 10, inp_freq, out_loan_value_perc,
                            out_cashflow, out_num_payments, 0.0000001, 1000)
#print ("Return of the loan:", out_loan_return)   


#############################################################


# Calculate the loan value without risk
out_loan_value_wr_perc = Final_LoanPrice_WR(inp_freq, inp_risk_class, inp_probability_matrix, inp_funding_cost_spread,
                                            out_num_payments, out_discount_rate, out_cashflow)
print ('Loan Value Without Risk (percentage):', out_loan_value_wr_perc)

# Calculate the loan value amount without risk
out_loan_value_wr_amt = out_loan_value_wr_perc * inp_loan_amount
#print ('Loan Value Amount Without Risk:', out_loan_value_wr_amt)

# Calculate the return of a loan without risk
out_return_without_risk = bisection(-10, 10, inp_freq, out_loan_value_wr_perc,
                                    out_cashflow, out_num_payments, 0.0000001, 1000)
print ("Return of the loan without risk:", out_return_without_risk)  

#####################################################################

# Calculate the loan value without risk
out_loan_value_wr_perc_hc = Final_LoanPrice_WR_HC(inp_freq, inp_risk_class, inp_probability_matrix, inp_funding_cost_spread,
                                            out_hedging_cost_perc, out_num_payments, out_discount_rate, out_cashflow_hc, inp_type_coupon) 
#print ('Loan Value Without Risk and hedging cost (percentage):', out_loan_value_wr_perc_hc)

# Calculate the loan value amount without risk

out_loan_value_wr_hc_perc = out_loan_value_wr_perc_hc - out_loan_value_wr_perc 
#print ('Loan Value Without Risk and hedging cost:', out_loan_value_wr_hc_perc)

out_loan_value_wr_hc_amt = (1- inp_type_coupon) * (out_loan_value_wr_perc - out_loan_value_wr_perc_hc) * inp_loan_amount   
#print ('Loan Value Amount Without Risk and hedging cost:', out_loan_value_wr_hc_amt)   

#######################################################################


# to calculate the operational cost 

out_loan_value_OC_perc = Final_LoanPrice_OC(inp_freq, inp_risk_class,  
                                            inp_probability_matrix, inp_funding_cost_spread, inp_operation_cost_spread, 
                                            out_num_payments, out_discount_rate, out_cashflow_oc)
#print ('the amount of operational cost (percentage):', out_loan_value_OC_perc) 

# Calculate the loan value amount without risk
out_loan_value_OC_amt = out_loan_value_OC_perc * inp_loan_amount
#print ('Loan Value Amount Without Risk and With Operational Cost:', out_loan_value_OC_amt)



amount_of_operational_cost = out_loan_value_wr_amt - out_loan_value_OC_amt     
#print ('Loan Value Amount With Operational Cost:', amount_of_operational_cost) 

###################################################################################


# Calculate the loan value without funding cost 

out_loan_value_wfc_perc = Final_LoanPrice_WFC(inp_freq, inp_risk_class, inp_probability_matrix, inp_funding_cost_spread,
                                            out_num_payments, out_discount_rate, out_cashflow)   
print ('Loan Value Without Funding Cost (percentage):', out_loan_value_wfc_perc) 

# Calculate the loan value amount without risk
out_loan_value_wfc_amt = out_loan_value_wfc_perc * inp_loan_amount
#print ('Loan Value Amount Without Risk and Without FC:', out_loan_value_wfc_amt)

# Calculate the return of a loan without risk
out_return_without_funding_cost = bisection(-10, 10, inp_freq, out_loan_value_wfc_perc, 
                                    out_cashflow, out_num_payments, 0.0000001, 1000)
print ("Return of the loan without funding cost:", out_return_without_funding_cost) 


# Calculate the amount of funding cost 

amount_of_funding_cost = out_loan_value_wfc_amt - out_loan_value_wr_amt   
#print ('Loan Value Amount Without FC:', amount_of_funding_cost)

####################################################################################


# Calculate the expected loss (percentage)
out_expected_loss_perc = Final_ExpectedLoss(inp_freq, inp_lgd, inp_risk_class, inp_probability_matrix, 
                                              inp_funding_cost_spread, out_discount_rate, out_notional_value, 
                                              out_interest_payment, out_num_payments) 
#print ('Expected Loss (percentage):', out_expected_loss_perc)

# Calculate the expected loss amount
out_expected_loss_amt = out_expected_loss_perc * inp_loan_amount
#print ('Expected Loss Amount:', out_expected_loss_amt) 


#########################################################################################


out_pd = PD(inp_probability_matrix, inp_risk_class)
#print ('Probability of Default:', out_pd)  


# Calculate the economic capital (percentage)
out_economic_capital_perc = EconomicCapital(out_pd, const_init_notional_val, inp_lgd, const_rho, const_aa)
#print ('Economic Capital (percentage):', out_economic_capital_perc)

# Calculate the economic capital amount    
out_economic_capital_amt = out_economic_capital_perc * inp_loan_amount
#print ('Economic Capital Amount:', out_economic_capital_amt)


# Calculate the unexpected loss
out_unexpected_loss_perc = Unexpectedloss(out_economic_capital_perc, const_wt, const_wr, const_nd)
#print ('Unexpected Loss (percentage):', out_unexpected_loss_perc)

# Calculate the unexpected loss amount   
out_unexpected_loss_amt = out_unexpected_loss_perc * inp_loan_amount
#print ('Unexpected Loss Amount:', out_unexpected_loss_amt) 


####################################################################


# Expected Loss Return
out_expected_loss_return = bisection(-10, 10, inp_freq, out_loan_value_wr_perc - out_expected_loss_perc,
                                     out_cashflow, out_num_payments, 0.0000001, 1000)
#print ("Return of the expected loss:", out_expected_loss_return)   

# Unexpected Loss Return
out_unexpected_loss_return = bisection(-10, 10, inp_freq, out_loan_value_wr_perc - out_unexpected_loss_perc,
                                       out_cashflow, out_num_payments, 0.0000001, 1000)
#print ("Return of the unexpected loss:", out_unexpected_loss_return)

# Hedging Cost Return
out_hedging_cost_return = bisection(-10, 10, inp_freq, out_loan_value_wr_perc - out_hedging_cost_perc,
                                    out_cashflow, out_num_payments, 0.0000001, 1000)
#print ("Return of the hedging cost:", out_hedging_cost_return)

########################################################################


# Expected loss spread
out_expected_loss_spread = SpreadOf(out_expected_loss_return, out_return_without_risk)
#print ("Expected loss spread:", out_expected_loss_spread)

# Unexpected loss spread
out_unexpected_loss_spread = SpreadOf(out_unexpected_loss_return, out_return_without_risk)
#print ("Unexpected loss spread:", out_unexpected_loss_spread)

# Hedging Cost spread
out_hedging_cost_spread = SpreadOf(out_hedging_cost_return, out_return_without_risk)
#print ("Hedging cost spread:", out_hedging_cost_spread) 


############################################################################

# calculation for the RAROC formula    

OUT_raroc, out_hurdle_rate = RAROC(out_loan_return, out_return_without_funding_cost, inp_loan_spread, inp_fixed_rate, inp_type_coupon, out_hedging_cost_perc, inp_funding_cost_spread, 
                  out_expected_loss_spread, inp_operation_cost_spread, out_unexpected_loss_spread, 
                  out_economic_capital_perc, const_init_notional_val, const_wr)
#print ("hurdle_rate:", out_hurdle_rate)   
#print ("RAROC:", OUT_raroc) 

#############################################################################


# Record the date and time of the end of the process  
test_end_date = datetime.datetime.now()

# Calculate the duration of the process
test_duration = test_end_date - test_start_date

#print ('The process run in ' + str(test_duration))    

Number of Payments: 10
Loan Value Without Risk (percentage): 1.0095891879776067
Return of the loan without risk: 0.004918202757835388
Loan Value Without Funding Cost (percentage): 1.0234929388353469
Return of the loan without funding cost: -7.681548595428467e-05


In [25]:
########################################################################
# PANDAS SUPPORT
########################################################################

In [26]:
def pandas_pricing_algorithm(df, cfg_probability_matrix, cfg_tenor, cfg_swap_rates, cfg_funding_cost_spread, cfg_redemption_rate):

    # Record the date and time of the beginning of the process
    algorithm_start_date = datetime.datetime.now()

    df_res = df.copy()

    # Aux calculations
    df_res['maturity_years'] = df_res.apply(lambda r: int(r['maturity'] / 12.0), axis = 1)
    df_res['risk_class_zb'] = df_res.apply(lambda r: r['risk_class'] - 1, axis = 1)
    
    # Start algorithm

    df_res['out_num_payments'] = df_res.apply(lambda r: NumberofPayment(
        r['payment_freq'], r['maturity_years']), axis = 1)

    df_res['out_amortisation'] = df_res.apply(lambda r: Amortisation(
        const_init_notional_val, r['out_num_payments']), axis = 1)

    df_res['out_notional_value'] = df_res.apply(lambda r: Notional_Value(
        const_init_notional_val, r['out_num_payments'], r['out_amortisation']), axis = 1)

    df_res['out_int_zero_rate'] = df_res.apply(lambda r: Rate_Aux(
        r['payment_freq'], r['interest_rate_index'], cfg_tenor, cfg_swap_rates, r['out_num_payments']), axis = 1)

    df_res['out_forward_rate'] = df_res.apply(lambda r: Forward_Rate(
        r['out_int_zero_rate'], r['interest_rate_index'], r['out_num_payments']), axis = 1)
    
    df_res['out_discount_rate'] = df_res.apply(lambda r: Discount_Rate(
        r['out_int_zero_rate'], r['out_num_payments']), axis = 1)
    
    df_res['out_cashflow_instalment'] = df_res.apply(lambda r: Cashflow_Instalment(
        r['payment_freq'], const_init_notional_val, r['spread'], r['fixed_rate'], r['type_coupon'], 
        cfg_redemption_rate, r['out_num_payments'], r['out_amortisation'], r['out_forward_rate']), axis = 1)
    
    df_res['out_interest_payment'] = df_res.apply(lambda r: Interest_Payment(
        r['payment_freq'], const_init_notional_val, r['spread'], r['fixed_rate'], r['type_coupon'],
        cfg_redemption_rate, r['out_num_payments'], r['out_amortisation'], r['out_forward_rate']), axis = 1)
    
    df_res['out_pd'] = df_res.apply(lambda r: PD(
        cfg_probability_matrix, r['risk_class_zb']), axis = 1)
    
    df_res['out_loan_value_perc'] = df_res.apply(lambda r: Final_LoanPrice(
        r['payment_freq'], r['lgd'], r['risk_class_zb'], cfg_probability_matrix, cfg_funding_cost_spread,
        r['out_num_payments'], r['out_discount_rate'], r['out_notional_value'], r['out_cashflow_instalment'],
        r['out_interest_payment']), axis = 1)

    df_res['out_loan_value_amt'] = df_res.apply(lambda r: r['out_loan_value_perc'] * r['loan_amount'], axis = 1)
    
    df_res['out_loan_value_wr_perc'] = df_res.apply(lambda r: Final_LoanPrice_WR(
        r['payment_freq'], r['risk_class_zb'], cfg_probability_matrix, cfg_funding_cost_spread,
        r['out_num_payments'], r['out_discount_rate'], r['out_cashflow_instalment']), axis = 1)
    
    df_res['out_loan_value_wr_amt'] = df_res.apply(lambda r: r['out_loan_value_wr_perc'] * r['loan_amount'], axis = 1)
    
    df_res['out_hedging_cost_perc'] = df_res.apply(lambda r: HedgingCost(
        r['payment_freq'], r['out_notional_value'], r['out_forward_rate'], r['spread'], r['out_num_payments']), axis = 1)

    df_res['out_hedging_cost_amt'] = df_res.apply(lambda r: r['out_hedging_cost_perc'] * r['loan_amount'], axis = 1)
    
    df_res['out_economic_capital_perc'] = df_res.apply(lambda r: EconomicCapital(
        r['out_pd'], const_init_notional_val, r['lgd'], const_rho, const_aa), axis = 1)
    
    df_res['out_economic_capital_amt'] = df_res.apply(lambda r: r['out_economic_capital_perc'] * r['loan_amount'], axis = 1)
    
    df_res['out_expected_loss_perc'] = df_res.apply(lambda r: Final_ExpectedLoss(
        r['payment_freq'], r['lgd'], r['risk_class_zb'], cfg_probability_matrix, r['out_discount_rate'],
        r['out_notional_value'], r['out_interest_payment'], r['out_num_payments']), axis = 1)
    
    df_res['out_expected_loss_amt'] = df_res.apply(lambda r: r['out_expected_loss_perc'] * r['loan_amount'], axis = 1)
    
    df_res['out_unexpected_loss_perc'] = df_res.apply(lambda r: Unexpectedloss(
        r['out_economic_capital_perc'], const_wt, const_wr, const_nd), axis = 1)
    
    df_res['out_unexpected_loss_amt'] = df_res.apply(lambda r: r['out_unexpected_loss_perc'] * r['loan_amount'], axis = 1)
    
    df_res['out_loan_return'] = df_res.apply(lambda r: bisection(
        -10, 10, r['payment_freq'], r['out_loan_value_perc'], r['out_cashflow_instalment'],
        r['out_num_payments'], 0.0000001, 1000), axis = 1)
    
    df_res['out_return_without_risk'] = df_res.apply(lambda r: bisection(
        -10, 10, r['payment_freq'], r['out_loan_value_wr_perc'], r['out_cashflow_instalment'],
        r['out_num_payments'], 0.0000001, 1000), axis = 1)
    
    #df_res['out_return_without_risk_1'] = df_res.apply(lambda r: bisection(
    # -10, 10, r['payment_freq'], 1, r['out_cashflow_instalment'],
    # r['out_num_payments'], 0.0000001, 1000), axis = 1)
    
    df_res['out_expected_loss_return'] = df_res.apply(lambda r: bisection(
        -10, 10, r['payment_freq'], r['out_loan_value_wr_perc'] - r['out_expected_loss_perc'],
        r['out_cashflow_instalment'], r['out_num_payments'], 0.0000001, 1000), axis = 1)
    
    df_res['out_unexpected_loss_return'] = df_res.apply(lambda r: bisection(
        -10, 10, r['payment_freq'], r['out_loan_value_wr_perc'] - r['out_unexpected_loss_perc'],
        r['out_cashflow_instalment'], r['out_num_payments'], 0.0000001, 1000), axis = 1)
    
    df_res['out_hedging_cost_return'] = df_res.apply(lambda r: bisection(
        -10, 10, r['payment_freq'], r['out_loan_value_wr_perc'] - r['out_hedging_cost_perc'],
        r['out_cashflow_instalment'], r['out_num_payments'], 0.0000001, 1000), axis = 1)
    
    df_res['out_expected_loss_spread'] = df_res.apply(lambda r: SpreadOf(
        r['out_expected_loss_return'], r['out_return_without_risk']), axis = 1)
    
    df_res['out_unexpected_loss_spread'] = df_res.apply(lambda r: SpreadOf(
        r['out_unexpected_loss_return'], r['out_return_without_risk']), axis = 1)
    
    df_res['out_hedging_cost_spread'] = df_res.apply(lambda r: SpreadOf(
        r['out_hedging_cost_return'], r['out_return_without_risk']), axis = 1)

    # Record the date and time of the end of the process
    algorithm_end_date = datetime.datetime.now()

    df_res['start_date'] = algorithm_start_date
    df_res['end_date'] = algorithm_end_date

    # Calculate the duration of the process
    algorithm_duration = algorithm_end_date - algorithm_start_date

    print ('The process run in ' + str(algorithm_duration))